# Notebook for Feature Engineering

This notebook will contain a few steps to get the data ready for model training

1. Data Cleaning (removing nulls, and other bad data)
2. Feature Engineering (creating target features AUC+IC50)
3. Creating Train Test Splits (including leave out drugs and leave out cell lines)

In [ ]:
import pandas as pd

# load in the merged dataframe, containing GDSC1, GDSC2, and cell line metadata
gdsc_merged = pd.read_csv("../GDSC1and2_w_CellLineData.csv")
df = gdsc_merged
gdsc_merged.head(5) 

In [ ]:
gdsc_merged[gdsc_merged['DRUG_NAME'] == 'Etoposide']

## Feature Engineering

## Creation of Interaction Term IC50 + AUC

In this segment we create sensitivity, disagreement and weighted averages for these terms. The goal is to have two metric targets and be able to train with either a dual output approach or a singlet output with weighted averages. I'll let the training team decided which to use or go with the one with better performance.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[["z_LN_IC50", "z_AUC"]] = scaler.fit_transform(df[["LN_IC50", "AUC"]])

# For interpretability: low LN_IC50 = sensitive, low AUC = sensitive
df["z_IC50_sens"] = df["z_LN_IC50"]


# Sensitivity (average of both)
df["sensitivity"] = (df["z_IC50_sens"] + df["z_AUC"]) / 2

# Disagreement (difference between metrics)
df["disagreement"] = df["z_AUC"] - df["z_IC50_sens"]


# Weighted averages of both metrics for different α
alphas = [0.25, 0.5, 0.75]
for a in alphas:
    df[f"y_weighted_{a}"] = a * df["z_IC50_sens"] + (1 - a) * df["z_AUC"]

df.head(5)

## Tissue Descriptors

Lets focus on using only Tissue Descriptor 1 "GDSC_Tissue_descriptor_1". I would rather lean on the larger sample size of each bin within Tissue Descriptor 1 for ease/simplicity. I think there is a good argument for usind Tissue descriptor 2 for personalized medicine and specific drug discovery but its not worth it for now. We can't do a simple bootstrapping method to bring up categories with 5 samples up to 60 without creating significant bias. Tissue Descriptor 1 still has imbalances but the effects won't be as severe. It would be reasonable to combine oversampling and undersampling in this case though.

## Data Splits for Model Training
1
Random splits: This approach is also called Mixed-Set in [8, 39], and it is generally the least challenging, leading to the highest observed performance scores. In this scenario, a randomly selected subset of drug-cell line pairs is excluded from the training set and used as the test set. This train-test Splitting Strategy quantifies how accurate a model is in filling the gaps in a drug-cell lines matrix containing some unobserved values. Practically, this would correspond to filling a non-exhaustive screening on a panel of otherwise known cell lines and drugs. In this scenario, the model is not evaluated in terms of its ability to generalize to cell lines or drugs for which we completely lack drug response measurements.

2
Unseen cell lines: In this case, the train and test splits are made by ensuring that the cell lines in the training set are not present in the test. The test set is constructed by randomly selecting a subset of cell lines and all of their IC50 values from the entire dataset. To achieve high performance scores in this validation, the models need to be able to generalize to unseen cell lines. With respect to the Random Splits, this therefore increases the difficulty of the prediction task.

3
Unseen drugs: The train and test splits are made to ensure that the drugs that appear in the test set are not present in the training set. To perform well in this setting, the model must be able to generalize well to completely unseen drugs.

4
Unseen cell line-drug pairs: This is the most stringent validation setting. In this case, the training and test splits are built to ensure that each of the cell lines and drugs present in the test set are both absent from the training set. This setting therefore evaluates the ability of the model to generalize at the same time to unseen drugs and cell lines, which should be the ultimate goal of the cancer drug sensitivity prediction field. However, until now, generalization in this setting has been nearly impossible, and as such, it is infrequently utilized in evaluations [9].

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def random_split(df, test_size=0.2, random_state=42):
    """
    Completely random splitting.
    """
    return train_test_split(df, test_size=test_size, random_state=random_state, shuffle=True)

def unseen_cell_lines_split(df, test_size=0.2, random_state=42):
    """
    Splits dataset so that cell lines in test are unseen in train.
    """
    cell_lines = df['COSMIC_ID'].unique()
    train_lines, test_lines = train_test_split(cell_lines, test_size=test_size, random_state=random_state)
    train_df = df[df['COSMIC_ID'].isin(train_lines)]
    test_df = df[df['COSMIC_ID'].isin(test_lines)]
    return train_df, test_df

def unseen_drugs_split(df, test_size=0.2, random_state=1):
    """
    Splits dataset so that drugs in test are unseen in train.
    """
    drugs = df['DRUG_ID'].unique()
    train_drugs, test_drugs = train_test_split(drugs, test_size=test_size, random_state=random_state)
    train_df = df[df['DRUG_ID'].isin(train_drugs)]
    test_df = df[df['DRUG_ID'].isin(test_drugs)]
    return train_df, test_df

def unseen_cell_line_drug_pairs_split(df, test_size=0.2, random_state=42):
    """
    Creates disjoint sets of both drugs and cell lines.
    Test set contains combinations of unseen drugs and unseen cell lines.
    """
    drugs = df['DRUG_ID'].unique()
    cell_lines = df['COSMIC_ID'].unique()

    # Select subsets of drugs and cell lines for the test set
    test_drugs = np.random.default_rng(random_state).choice(drugs, size=int(len(drugs) * test_size), replace=False)
    test_cells = np.random.default_rng(random_state + 1).choice(cell_lines, size=int(len(cell_lines) * test_size), replace=False)

    # Test = only pairs where BOTH drug and cell line are unseen
    test_df = df[(df['DRUG_ID'].isin(test_drugs)) & (df['COSMIC_ID'].isin(test_cells))]

    # Train = everything else (ensures no leakage)
    train_df = df[~df.index.isin(test_df.index)]

    return train_df, test_df

In [ ]:
train_df, test_df = unseen_drugs_split(df)
train_df.head(5)

In [ ]:
test = set(test_df['DRUG_ID'].unique()) 
train = set(train_df['DRUG_ID'].unique())

test.intersection(train)
# Confirm no intersection in left out sets.

## Training and Evaluation of the model

We train Random Forest models using random split (mixed-set approach). This is the least challenging splitting strategy where random drug-cell line pairs are used for train/test split, allowing the model to learn from similar drugs and cell lines.

## Random Splitting

In [ ]:
# Train test split with random splitting
train_df_random, test_df_random = random_split(df)

print(f"Training samples: {len(train_df_random)}")
print(f"Test samples: {len(test_df_random)}")
print(f"\nSample overlap statistics:")
print(f"Unique drugs in train: {train_df_random['DRUG_ID'].nunique()}")
print(f"Unique drugs in test: {test_df_random['DRUG_ID'].nunique()}")
print(f"Unique cell lines in train: {train_df_random['COSMIC_ID'].nunique()}")
print(f"Unique cell lines in test: {test_df_random['COSMIC_ID'].nunique()}")

## Data encoding and feature preparation

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Make explicit copies to avoid SettingWithCopyWarning
# train_df_random = train_df_random.copy()
# test_df_random = test_df_random.copy()

# # Encode categorical variables (DRUG_ID and COSMIC_ID)
# drug_encoder = LabelEncoder()
# cell_encoder = LabelEncoder()

# train_df_random['DRUG_ID_encoded'] = drug_encoder.fit_transform(train_df_random['DRUG_ID'])
# train_df_random['COSMIC_ID_encoded'] = cell_encoder.fit_transform(train_df_random['COSMIC_ID'])

# # For random split, test set should have mostly seen drugs/cells, so direct transform is usually safe
# test_df_random['DRUG_ID_encoded'] = drug_encoder.transform(test_df_random['DRUG_ID'])
# test_df_random['COSMIC_ID_encoded'] = cell_encoder.transform(test_df_random['COSMIC_ID'])

# Prepare features and target
X_train = train_df_random[['DRUG_ID', 'COSMIC_ID']]
y_train = train_df_random['z_LN_IC50']

X_test = test_df_random[['DRUG_ID', 'COSMIC_ID']]
y_test = test_df_random['z_LN_IC50']

## Model Training for z_LN_IC50 Prediction

Fitting modes to predict z_LN_IC50 

### Random Forest and Hyper Parameter Tuning for z_LN_IC50

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20, None]
}

# Create Random Forest model
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

# Perform Grid Search with cross-validation
print("Performing Grid Search for Random Forest hyperparameters...")
print("This may take a few minutes...")

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

# Print best parameters
print(f"\nBest parameters found:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest cross-validation RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

# Use the best model
rf_model = grid_search.best_estimator_

# Make predictions
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Evaluate model
train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf = r2_score(y_test, y_test_pred_rf)

print("Random Forest Model Performance:")
print(f"Train RMSE: {train_rmse_rf:.4f}")
print(f"Test RMSE: {test_rmse_rf:.4f}")
print(f"Train R²: {train_r2_rf:.4f}")
print(f"Test R²: {test_r2_rf:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': ['DRUG_ID', 'COSMIC_ID'],
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)
print(f"\nFeature Importance:")
print(feature_importance)

# Create scatter plot of actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred_rf, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual z_LN_IC50', fontsize=12)
plt.ylabel('Predicted z_LN_IC50', fontsize=12)
plt.title('Random Forest: Actual vs Predicted Values (Test Set)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print additional statistics
residuals_rf = y_test - y_test_pred_rf
print(f"\nResiduals Statistics:")
print(f"Mean Residual: {residuals_rf.mean():.4f}")
print(f"Std Residual: {residuals_rf.std():.4f}")
print(f"Min Residual: {residuals_rf.min():.4f}")
print(f"Max Residual: {residuals_rf.max():.4f}")

### Save Random Forest model for ln(IC50) prediction as .pkl file to be used in Streamlit app

In [ ]:
import joblib

joblib.dump(rf_model, 'rf_ic50.pkl', compress=3)

# Model Training for z_AUC Prediction

Now we train all models to predict z_AUC instead of z_LN_IC50

### Data preparation for z_AUC prediction

In [ ]:
# Prepare target variable for z_AUC prediction
y_train_auc = train_df_random['z_AUC']
y_test_auc = test_df_random['z_AUC']

print(f"Training samples: {len(y_train_auc)}")
print(f"Test samples: {len(y_test_auc)}")

### Random Forest and Hyper Parameter Tuning for z_AUC

In [ ]:
# Random Forest for z_AUC
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20, None]
}

rf_base_auc = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search_rf_auc = GridSearchCV(rf_base_auc, param_grid_rf, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search_rf_auc.fit(X_train, y_train_auc)

print(f"Best params: {grid_search_rf_auc.best_params_}")
print(f"Best CV RMSE: {np.sqrt(-grid_search_rf_auc.best_score_):.4f}")

rf_model_auc = grid_search_rf_auc.best_estimator_
y_train_pred_rf_auc = rf_model_auc.predict(X_train)
y_test_pred_rf_auc = rf_model_auc.predict(X_test)

train_rmse_rf_auc = np.sqrt(mean_squared_error(y_train_auc, y_train_pred_rf_auc))
test_rmse_rf_auc = np.sqrt(mean_squared_error(y_test_auc, y_test_pred_rf_auc))
train_r2_rf_auc = r2_score(y_train_auc, y_train_pred_rf_auc)
test_r2_rf_auc = r2_score(y_test_auc, y_test_pred_rf_auc)

print("Random Forest (z_AUC):")
print(f"Train RMSE: {train_rmse_rf_auc:.4f}, Test RMSE: {test_rmse_rf_auc:.4f}")
print(f"Train R²: {train_r2_rf_auc:.4f}, Test R²: {test_r2_rf_auc:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_test_auc, y_test_pred_rf_auc, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_test_auc.min(), y_test_auc.max()], [y_test_auc.min(), y_test_auc.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual z_AUC', fontsize=12)
plt.ylabel('Predicted z_AUC', fontsize=12)
plt.title('Random Forest: z_AUC Prediction', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Save Random Forest model for z_AUC prediction as .pkl file

In [ ]:
joblib.dump(rf_model_auc, 'rf_auc.pkl', compress=3)

# Model Training for y_weighted_0.25 Prediction

Now we train all models to predict y_weighted_0.25 (weighted combination: 0.25 * z_LN_IC50 + 0.75 * z_AUC)

### Data preparation for y_weighted_0.25 prediction

In [ ]:
# Prepare target variable for y_weighted_0.25 prediction
y_train_weighted = train_df_random['y_weighted_0.25']
y_test_weighted = test_df_random['y_weighted_0.25']

print(f"Training samples: {len(y_train_weighted)}")
print(f"Test samples: {len(y_test_weighted)}")
print(f"\nTarget statistics:")
print(f"Mean: {y_train_weighted.mean():.4f}")
print(f"Std: {y_train_weighted.std():.4f}")

### Random Forest and Hyper Parameter tuning for y_weighted_0.25

In [ ]:
# Random Forest for y_weighted_0.25
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20, None]
}

rf_base_weighted = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search_rf_weighted = GridSearchCV(rf_base_weighted, param_grid_rf, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search_rf_weighted.fit(X_train, y_train_weighted)

print(f"Best params: {grid_search_rf_weighted.best_params_}")
print(f"Best CV RMSE: {np.sqrt(-grid_search_rf_weighted.best_score_):.4f}")

rf_model_weighted = grid_search_rf_weighted.best_estimator_
y_train_pred_rf_weighted = rf_model_weighted.predict(X_train)
y_test_pred_rf_weighted = rf_model_weighted.predict(X_test)

train_rmse_rf_weighted = np.sqrt(mean_squared_error(y_train_weighted, y_train_pred_rf_weighted))
test_rmse_rf_weighted = np.sqrt(mean_squared_error(y_test_weighted, y_test_pred_rf_weighted))
train_r2_rf_weighted = r2_score(y_train_weighted, y_train_pred_rf_weighted)
test_r2_rf_weighted = r2_score(y_test_weighted, y_test_pred_rf_weighted)

print("Random Forest (y_weighted_0.25):")
print(f"Train RMSE: {train_rmse_rf_weighted:.4f}, Test RMSE: {test_rmse_rf_weighted:.4f}")
print(f"Train R²: {train_r2_rf_weighted:.4f}, Test R²: {test_r2_rf_weighted:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_test_weighted, y_test_pred_rf_weighted, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_test_weighted.min(), y_test_weighted.max()], [y_test_weighted.min(), y_test_weighted.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual y_weighted_0.25', fontsize=12)
plt.ylabel('Predicted y_weighted_0.25', fontsize=12)
plt.title('Random Forest: y_weighted_0.25 Prediction', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Model Training for Sensitivity Prediction

Now we train all models to predict sensitivity (average of z_IC50_sens and z_AUC)

### Data preparation for sensitivity prediction

In [ ]:
# Prepare target variable for sensitivity prediction
y_train_sens = train_df_random['sensitivity']
y_test_sens = test_df_random['sensitivity']

print(f"Training samples: {len(y_train_sens)}")
print(f"Test samples: {len(y_test_sens)}")
print(f"\nTarget statistics:")
print(f"Mean: {y_train_sens.mean():.4f}")
print(f"Std: {y_train_sens.std():.4f}")

### Random Forest and Hyper Parameter Tuning for sensitivity

In [ ]:
# Random Forest for sensitivity
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20, None]
}

rf_base_sens = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search_rf_sens = GridSearchCV(rf_base_sens, param_grid_rf, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search_rf_sens.fit(X_train, y_train_sens)

print(f"Best params: {grid_search_rf_sens.best_params_}")
print(f"Best CV RMSE: {np.sqrt(-grid_search_rf_sens.best_score_):.4f}")

rf_model_sens = grid_search_rf_sens.best_estimator_
y_train_pred_rf_sens = rf_model_sens.predict(X_train)
y_test_pred_rf_sens = rf_model_sens.predict(X_test)

train_rmse_rf_sens = np.sqrt(mean_squared_error(y_train_sens, y_train_pred_rf_sens))
test_rmse_rf_sens = np.sqrt(mean_squared_error(y_test_sens, y_test_pred_rf_sens))
train_r2_rf_sens = r2_score(y_train_sens, y_train_pred_rf_sens)
test_r2_rf_sens = r2_score(y_test_sens, y_test_pred_rf_sens)

print("Random Forest (sensitivity):")
print(f"Train RMSE: {train_rmse_rf_sens:.4f}, Test RMSE: {test_rmse_rf_sens:.4f}")
print(f"Train R²: {train_r2_rf_sens:.4f}, Test R²: {test_r2_rf_sens:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_test_sens, y_test_pred_rf_sens, alpha=0.5, edgecolors='k', linewidth=0.5)
plt.plot([y_test_sens.min(), y_test_sens.max()], [y_test_sens.min(), y_test_sens.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual sensitivity', fontsize=12)
plt.ylabel('Predicted sensitivity', fontsize=12)
plt.title('Random Forest: sensitivity Prediction', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import joblib
joblib.dump(rf_model_sens, 'rf_sens.pkl', compress=3)

## Model Performance Summary

Comparison of Random Forest for all target variables

In [ ]:
# Create comprehensive summary table for Random Forest only
summary_data = {
    'Target': ['z_LN_IC50', 'z_AUC', 'y_weighted_0.25', 'sensitivity'],
    'Train RMSE': [
        train_rmse_rf,
        train_rmse_rf_auc,
        train_rmse_rf_weighted,
        train_rmse_rf_sens
    ],
    'Test RMSE': [
        test_rmse_rf,
        test_rmse_rf_auc,
        test_rmse_rf_weighted,
        test_rmse_rf_sens
    ],
    'Train R²': [
        train_r2_rf,
        train_r2_rf_auc,
        train_r2_rf_weighted,
        train_r2_rf_sens
    ],
    'Test R²': [
        test_r2_rf,
        test_r2_rf_auc,
        test_r2_rf_weighted,
        test_r2_rf_sens
    ]
}

summary_df = pd.DataFrame(summary_data)

# Round values for better readability
summary_df['Train RMSE'] = summary_df['Train RMSE'].round(4)
summary_df['Test RMSE'] = summary_df['Test RMSE'].round(4)
summary_df['Train R²'] = summary_df['Train R²'].round(4)
summary_df['Test R²'] = summary_df['Test R²'].round(4)

print("=" * 80)
print("RANDOM FOREST MODEL PERFORMANCE SUMMARY - Random Split")
print("=" * 80)
print()
print(summary_df.to_string(index=False))
print()

# Find best performing target
best_target = summary_df.loc[summary_df['Test RMSE'].idxmin()]
print("=" * 80)
print("BEST PERFORMING TARGET (Based on Test RMSE):")
print("=" * 80)
print(f"Target: {best_target['Target']}")
print(f"Test RMSE: {best_target['Test RMSE']:.4f}")
print(f"Test R²: {best_target['Test R²']:.4f}")
print("=" * 80)

### Convert pkl files to onnx for environment compatibility

In [ ]:
import joblib
import numpy as np
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# load existing sklearn model
rf_ic50 = joblib.load("../rf_ic50.pkl")  
initial_type = [('input', FloatTensorType([None, 2]))]

# convert to ONNX
onnx_model = convert_sklearn(rf_ic50, initial_types=initial_type)

# save ONNX model
with open("rf_ic50.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [ ]:
# load existing sklearn model
rf_auc = joblib.load("../rf_auc.pkl")  
initial_type = [('input', FloatTensorType([None, 2]))]

# convert to ONNX
onnx_model = convert_sklearn(rf_auc, initial_types=initial_type)

# save ONNX model
with open("rf_auc.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
# load existing sklearn model
rf_sens = joblib.load("../rf_sens.pkl")  
initial_type = [('input', FloatTensorType([None, 2]))]

# convert to ONNX
onnx_model = convert_sklearn(rf_sens, initial_types=initial_type)

# save ONNX model
with open("rf_sens.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())